In [2]:
from mne_bids import BIDSPath, write_raw_bids, get_anonymization_daysback
import random
import pandas as pd
import numpy as np
import itertools
import mne
import os
import sys
import re

In [27]:
import glob
for file in glob.glob('../data/raw/*vhdr'):
    print(file)

../data/raw/Subj10.vhdr


In [31]:
regex = re.compile(r'\d+')
sub = regex.findall(file)[0]
sub


'10'

In [24]:
RAW_DIR = '../data/raw/'
BIDS_DIR = '../data/bids/'
sub = '10'
run = '1'
task = 'expectationsABR'
fname = 'Subj10.vhdr'

# load data with MNE function for your file format
full_fpath = os.path.join(RAW_DIR, fname)
print(full_fpath)
raw = mne.io.read_raw_brainvision(full_fpath)
raw.load_data()

In [21]:
# add some info BIDS will want
print("Add line_freq to raw.info")
raw.info['line_freq'] = 60 # the power line frequency in the building we collected in

Add line_freq to raw.info


In [22]:
# Extract events from raw file
print("Set annotations")
events, event_ids = mne.events_from_annotations(raw)

# Drop meaningless event name
events = np.array(events)
events = events[events[:,2] != event_ids['New Segment/'], :]

Set annotations
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  7', 'Stimulus/S  8', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15']


In [25]:
# Get range of dates the BIDS specification will accept
daysback_min, daysback_max = get_anonymization_daysback(raw)

# Write data into BIDS directory, while anonymizing
print("Write data into BIDS directory")
bids_path = BIDSPath(
        run = run,
        subject = sub,
        task = task,
        datatype = 'eeg',
        root = BIDS_DIR
)

write_raw_bids(
    raw,
    bids_path = bids_path,
    allow_preload = True, # whether to load full dataset into memory when copying
    format = 'BrainVision', # format to save to
    anonymize = dict(daysback = daysback_min), # shift dates by daysback
    overwrite = True,
)

# Check if conversion was successful and .vhdr file was written
vhdr_path = str(bids_path)
print(f".vhdr file written? {os.path.exists(vhdr_path)}")
print("Done :-)")

Write data into BIDS directory
Writing '../data/bids/README'...
Writing '../data/bids/participants.tsv'...
Writing '../data/bids/participants.json'...
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  7', 'Stimulus/S  8', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15']
Writing '../data/bids/sub-10/eeg/sub-10_task-expectationsABR_run-1_events.tsv'...
Writing '../data/bids/dataset_description.json'...
Writing '../data/bids/sub-10/eeg/sub-10_task-expectationsABR_run-1_eeg.json'...
Writing '../data/bids/sub-10/eeg/sub-10_task-expectationsABR_run-1_channels.tsv'...
Copying data files to sub-10_task-expectationsABR_run-1_eeg.vhdr


/var/folders/bk/d0152l256wj91ds4hp514cj40000gn/T/ipykernel_2569/2079932072.py:14: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(
/Users/letitiaho/miniconda3/envs/pitch-tracking/lib/python3.10/site-packages/mne/utils/check.py:92: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion(library.__version__) < LooseVersion(min_version):
/Users/letitiaho/miniconda3/envs/pitch-tracking/lib/python3.10/site-packages/mne/utils/check.py:92: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LooseVersion(library.__version__) < LooseVersion(min_version):


Writing '../data/bids/sub-10/sub-10_scans.tsv'...
Wrote ../data/bids/sub-10/sub-10_scans.tsv entry with eeg/sub-10_task-expectationsABR_run-1_eeg.vhdr.
.vhdr file written? True
Done :-)
